# Step 2 - Data Wrangling Raw Data in Local Data Lake to Digestable Data  
Loading, merging, cleansing, unifying and wrangling Oracle OpenWorld & CodeOne Session Data from still fairly raw JSON files in the local datalake. 

The gathering of raw data from the (semi-)public API for the Session Catalog into a local data lake was discussed and performed  in <a href="./1-OOW2018 Session Catalog - retrieving raw session data in JSON files.ipynb">Notebook 1-OOW2018 Session Catalog - retrieving raw session data in JSON files</a>. The current notebook starts from the 44 raw JSON files in local directory `./data`.

This notebook describes how to load, combine and wrangle the data from these files. This notebook shows for example how to load and merge data from dozens of (same formatted) JSON files, discard undesired attributes, deduplicate the record set, derive attributes for easier business intelligence & machine learning and write the resulting data set to a single JSON file.

Steps in this notebook:
* Load and Merge from raw JSON
    * <a href="#deduplicate">discard redundant columns</a>
    * <a href="#deduplicate">deduplication</a> 
* <a href="#explore">Explore Data Frame</a>
* <a href="#enriching">Enrich Data</a>
* <a href="publish">Publish Wrangle Results</a>

The deliverable from this notebook is a single file `oow2018-sessions-wrangled.json` in the `datawarehouse` folder. This file contains unique, filtered, enriched data that is in a poper shape to perform further analysis on.

# Load and merge data from raw JSON files
This first section describes how the session data from Oracle OpenWorld 2018 is loaded from over 40 individual files with the raw JSON session data. These files are organized by session type and event (oow and codeone) - and have been produced by a different notebook (<a href="./1-OOW2018 Session Catalog - retrieving raw session data in JSON files.ipynb">Notebook 1-OOW2018 Session Catalog - retrieving raw session data in JSON files</a>) from the Oracle OpenWorld Session Catalog API.

The files are read into individual Pandas Data Frame objects. These Data Frames are concatenated. The end result from reading 44 files is a single Pandas Data Frame - called *ss* (session schedule).

Let's start with reading the session information from a single file into a Pandas Data Frame - to get a feel for how that works and what it results in.

In [166]:
#read a single session data file and parse the JSON content into a Pandas Data Frame
import pandas as pd
import json

dataLake = "datalake/" # file system directory used for storing the gathered data

#as a test, try to load data from one of the generated files 
conference = 'oow' # could also be codeone
sessionType = 'TRN' # could also be one of 21 other values such as TUT, DEV, GEN, BOF, HOL, ...
ss = pd.read_json("{0}oow2018-sessions_{1}_{2}.json".format(dataLake, conference, sessionType))
# add an additional column to the Data Frame to specify the conference catalog of origin of these sessions; in this case oow
ss = ss.assign(catalog=conference)

ss.head(3)


,abbreviation,abstract,allowDoubleBooking,attributevalues,code,codeParts,code_id,es_metadata_id,event,eventCode,...,type_displayorder_string,useDoubleBooking,useWaitingList,videos,viewAccess,viewAccessPublic,viewFileAccess,waitlistAccess,waitlistLimit,catalog
0,TRN4070,What if you could benefit from new database qu...,0,"[{'value': 'Beginner', 'attributevalue_id': '1...",TRN4070,"{'alpha0': 'TRN', 'numeric1': '4070'}",trn4070,1523650128750001VI1i,Oracle OpenWorld,oow18,...,9999,True,0,[],"[1533140082245001une2, 1533141196635001EZdJ, 1...",True,[],"[1533141196635001EZdJ, 1533143193464001uQsQ]",0,oow
1,TRN4026,How does a real-world performance engineer int...,0,"[{'value': 'Beginner', 'attributevalue_id': '1...",TRN4026,"{'alpha0': 'TRN', 'numeric1': '4026'}",trn4026,1523576222383001pdRl,Oracle OpenWorld,oow18,...,9999,True,0,[],"[1533140082245001une2, 1533141196635001EZdJ, 1...",True,[],"[1533141196635001EZdJ, 1533143193464001uQsQ]",0,oow
10,TRN4567,Understand the tools and techniques available ...,0,"[{'value': 'Beginner', 'attributevalue_id': '1...",TRN4567,"{'alpha0': 'TRN', 'numeric1': '4567'}",trn4567,1524514304383001hANs,Oracle OpenWorld,oow18,...,9999,True,0,[],"[1533140082245001une2, 1533141196635001EZdJ, 1...",True,[],"[1533141196635001EZdJ, 1533143193464001uQsQ]",0,oow


In [167]:
#as a test, try to load data from another file ; same sessionType but different conference 

conference = 'codeone' 
sessionType = 'TRN'
ss2 = pd.read_json("{0}oow2018-sessions_{1}_{2}.json".format(dataLake, conference, sessionType))
# add an additional column to the Data Frame to specify the conference catalog of origin of these sessions; in this case codeone
ss2 = ss2.assign(catalog='codeone')

ss2.head(3)


,abbreviation,abstract,allowDoubleBooking,attributevalues,code,codeParts,code_id,es_metadata_id,event,eventCode,...,type_displayorder_string,useDoubleBooking,useWaitingList,videos,viewAccess,viewAccessPublic,viewFileAccess,waitlistAccess,waitlistLimit,catalog
0,TRN4631,This session takes a deep dive into how the Or...,0,"[{'value': 'Beginner', 'attributevalue_id': '1...",TRN4631,"{'alpha0': 'TRN', 'numeric1': '4631'}",trn4631,1524589840877001mdKy,Oracle OpenWorld,oow18,...,9999,True,0,None,"[1533140082245001une2, 1533141196635001EZdJ, 1...",True,[],"[1533141196635001EZdJ, 1533143193464001uQsQ]",0,codeone
1,TRN4081,Real-time data is an increasingly popular topi...,0,"[{'value': 'Beginner', 'attributevalue_id': '1...",TRN4081,"{'alpha0': 'TRN', 'numeric1': '4081'}",trn4081,1523652232625001KxHD,Oracle OpenWorld,oow18,...,9999,True,0,[],"[1533140082245001une2, 1533141196635001EZdJ, 1...",True,[],"[1533141196635001EZdJ, 1533143193464001uQsQ]",0,codeone
10,TRN4084,.NET Core allows .NET applications to run cros...,0,"[{'value': 'Beginner', 'attributevalue_id': '1...",TRN4084,"{'alpha0': 'TRN', 'numeric1': '4084'}",trn4084,1523652635507001BgE3,Oracle OpenWorld,oow18,...,9999,True,0,[],"[1533140082245001une2, 1533141196635001EZdJ, 1...",True,[],"[1533141196635001EZdJ, 1533143193464001uQsQ]",0,codeone


All session data is to be merged into a single data frame. We will use `ss` as the sink - the data frame into which all sessions records are to be loaded. We use the Pandas concat operation to merge two Data Frames, as is done below for the two data frames with session records for session type TRN.

In [168]:
#add ss and ss2 together
#see https://pandas.pydata.org/pandas-docs/stable/merging.html
ss = pd.concat([ss,ss2],  ignore_index=True , sort=True)
ss.head(8)

,abbreviation,abstract,allowDoubleBooking,attributevalues,catalog,code,codeParts,code_id,es_metadata_id,event,...,type_displayorder,type_displayorder_string,useDoubleBooking,useWaitingList,videos,viewAccess,viewAccessPublic,viewFileAccess,waitlistAccess,waitlistLimit
0,TRN4070,What if you could benefit from new database qu...,0,"[{'value': 'Beginner', 'attributevalue_id': '1...",oow,TRN4070,"{'alpha0': 'TRN', 'numeric1': '4070'}",trn4070,1523650128750001VI1i,Oracle OpenWorld,...,9999,9999,True,0,[],"[1533140082245001une2, 1533141196635001EZdJ, 1...",True,[],"[1533141196635001EZdJ, 1533143193464001uQsQ]",0
1,TRN4026,How does a real-world performance engineer int...,0,"[{'value': 'Beginner', 'attributevalue_id': '1...",oow,TRN4026,"{'alpha0': 'TRN', 'numeric1': '4026'}",trn4026,1523576222383001pdRl,Oracle OpenWorld,...,9999,9999,True,0,[],"[1533140082245001une2, 1533141196635001EZdJ, 1...",True,[],"[1533141196635001EZdJ, 1533143193464001uQsQ]",0
2,TRN4567,Understand the tools and techniques available ...,0,"[{'value': 'Beginner', 'attributevalue_id': '1...",oow,TRN4567,"{'alpha0': 'TRN', 'numeric1': '4567'}",trn4567,1524514304383001hANs,Oracle OpenWorld,...,9999,9999,True,0,[],"[1533140082245001une2, 1533141196635001EZdJ, 1...",True,[],"[1533141196635001EZdJ, 1533143193464001uQsQ]",0
3,TRN6363,In order to derive value from the Internet of ...,0,"[{'value': 'Product Training Session', 'attrib...",oow,TRN6363,"{'alpha0': 'TRN', 'numeric1': '6363'}",trn6363,15321279159350012tnD,Oracle OpenWorld,...,9999,9999,True,0,[],"[1533140082245001une2, 1533141196635001EZdJ, 1...",True,[],"[1533141196635001EZdJ, 1533143193464001uQsQ]",0
4,TRN5828,Oracle Linux powers tens of thousands of enter...,0,"[{'value': 'Beginner', 'attributevalue_id': '1...",oow,TRN5828,"{'alpha0': 'TRN', 'numeric1': '5828'}",trn5828,1526317377636001I2Ll,Oracle OpenWorld,...,9999,9999,True,0,[],"[1533140082245001une2, 1533141196635001EZdJ, 1...",True,[],"[1533141196635001EZdJ, 1533143193464001uQsQ]",0
5,TRN4073,"What do Oracle Database listener, dedicated se...",0,"[{'value': 'Beginner', 'attributevalue_id': '1...",oow,TRN4073,"{'alpha0': 'TRN', 'numeric1': '4073'}",trn4073,1523650870534001B9V8,Oracle OpenWorld,...,9999,9999,True,0,None,"[1533140082245001une2, 1533141196635001EZdJ, 1...",True,[],"[1533141196635001EZdJ, 1533143193464001uQsQ]",0
6,TRN4022,Oracle Real Application Clusters (Oracle RAC) ...,0,"[{'value': 'Beginner', 'attributevalue_id': '1...",oow,TRN4022,"{'alpha0': 'TRN', 'numeric1': '4022'}",trn4022,1523572455776001BxBi,Oracle OpenWorld,...,9999,9999,True,0,[],"[1533140082245001une2, 1533141196635001EZdJ, 1...",True,[],"[1533141196635001EZdJ, 1533143193464001uQsQ]",0
7,TRN4219,"For more than 20 years, Oracle customers have ...",0,"[{'value': 'Beginner', 'attributevalue_id': '1...",oow,TRN4219,"{'alpha0': 'TRN', 'numeric1': '4219'}",trn4219,152392541435100177jo,Oracle OpenWorld,...,9999,9999,True,0,[],"[1533140082245001une2, 1533141196635001EZdJ, 1...",True,[],"[1533141196635001EZdJ, 1533143193464001uQsQ]",0


Overhead, two files were loaded, parsed into a Data Frame and added together into a single Data Frame. The next step is to load the session data from the raw JSON file for all 44 files - for the two events and for all session types. The session types are defined in the Dict object *sessionTypes* . The code loops over the keys in this Dict and reads the corresponding JSON file for each of the two events.

In [169]:
sessionTypes =  {'BOF': '1518466139979001dQkv'
    , 'BQS': 'bqs'
    , 'BUS': '1519240082595001EpMm'
    , 'CAS': 'casestudy'
    , 'DEV': '1522435540042001BxTD'
    , 'ESS': 'ess'
    , 'FLP': 'flp'
    , 'GEN': 'general'
    , 'HOL': 'hol'
    , 'HOM': 'hom'
    , 'IGN': 'ignite'
    , 'KEY': 'option_1508950285425'
    , 'MTE':'1523906206279002QAu9'
    , 'PKN': '1527614217434001RBfj'
    , 'PRO': '1518464344082003KVWZ'
    , 'PRM': '1518464344082002KM3k'
    , 'TRN': '1518464344082001KHky'
    , 'SIG': 'sig'
    , 'THT': 'ts'
    , 'TLD': '1537894888625001RriS'
    , 'TIP': '1517517756579001F3CR'
    , 'TUT': 'tutorial'
#commented out because TRN is dealt with earlier on, 'TRN': '1518464344082001KHky'
}
#loop over all session types and read the corresponding files for both events codeone and oow
for key,value in sessionTypes.items():
    sessionType = key
    conference = 'oow' 
    ssoow = pd.read_json("{0}oow2018-sessions_{1}_{2}.json".format(dataLake, conference, sessionType))
    # add an additional column to the Data Frame to specify the conference catalog of origin of these sessions
    ssoow = ssoow.assign(catalog=conference)
    
    conference = 'codeone' 
    sscodeone = pd.read_json("{0}oow2018-sessions_{1}_{2}.json".format(dataLake, conference, sessionType))
    sscodeone = sscodeone.assign(catalog=conference)
    
    # merge data for sessions of type session type for both oow and codeone into a master set in ss
    ss = pd.concat([ss,ssoow,sscodeone],  ignore_index=True, sort=True)
    
print("Done - all data is merged into one data frame")    

Done - all data is merged into one data frame


### Some key  metrics on the merged Sessions Set
The shape function on the data frame returns the dimensions of the frame: the number of rows by the number of columns:

In [170]:
ss.shape

(1991, 39)

In [171]:
# total memory usage
ss.memory_usage(index=True, deep=True).sum()

6816443

In [172]:
#list all columns in the Dataframe
ss.columns

Index(['abbreviation', 'abstract', 'allowDoubleBooking', 'attributevalues',
       'catalog', 'code', 'codeParts', 'code_id', 'es_metadata_id', 'event',
       'eventCode', 'eventId', 'eventName', 'externalID', 'featured_value',
       'files', 'highlight', 'length', 'participants',
       'publicViewPrivateSchedule', 'published', 'scheduleAccess', 'sessionID',
       'sponsors', 'status', 'times', 'title', 'title_sort', 'type',
       'type_displayorder', 'type_displayorder_string', 'useDoubleBooking',
       'useWaitingList', 'videos', 'viewAccess', 'viewAccessPublic',
       'viewFileAccess', 'waitlistAccess', 'waitlistLimit'],
      dtype='object')

In [173]:
# data types for the columns in the data frame
ss.dtypes

abbreviation                  object
abstract                      object
allowDoubleBooking           float64
attributevalues               object
catalog                       object
code                          object
codeParts                     object
code_id                       object
es_metadata_id                object
event                         object
eventCode                     object
eventId                       object
eventName                     object
externalID                    object
featured_value               float64
files                         object
highlight                     object
length                       float64
participants                  object
publicViewPrivateSchedule     object
published                    float64
scheduleAccess                object
sessionID                     object
sponsors                      object
status                        object
times                         object
title                         object
t

<a name="discard" />

## Discard unneeded attributes
Early inspection of the JSON document and the session catalog website has provided insight in the available attributes and their relevance. Below, you will see an overview of all columns in the Data Frame - corresponding with the top level items in the JSON document. The subsequent step removes from the Data Frame all columns that seem irrelevant for our task at hand. 

These columns seem relevant for the web frontend developers, for planned but not realized objectives or for unknown purposes. In order to not carry more weight than necessary - because of performance, resource usage and lurking complexity we get rid of columns that seem irrelevant. If we need them after all, they are still available in the data lake.    

Remove the unwanted columns from the Dataframe
(allowDoubleBooking .'codeParts', 'code_id', 'es_metadata_id',type_displayorder	type_displayorder_string	useDoubleBooking	useWaitingList	videos	viewAccess	viewAccessPublic, ...)


In [174]:
# remove columns
# Note that the original 'data' object is changed when inplace=True
ss.drop([
    'allowDoubleBooking'
         ,'codeParts', 'code_id', 'es_metadata_id','type_displayorder'
       ,'type_displayorder_string', 'useDoubleBooking'
       ,'useWaitingList', 'videos'
      , 'viewAccess', 'viewAccessPublic','viewFileAccess', 'waitlistAccess', 'waitlistLimit'
    ,  'eventId','eventName','featured_value','publicViewPrivateSchedule','published', 'scheduleAccess','sessionID','status'
    ,'externalID','highlight','abbreviation'
        ]
        , axis=1, inplace=True) 

<a name="deduplicate" />

## Deduplicate
Some sessions are included in the catalog for both events - Oracle OpenWorld and CodeOne - even though they are associated with one of the two. The exact same session - with only a different value for attribute catalog - occurs twice in our data set for these sessions. We should get rid of duplicates. However, we should not do so before we capture the fact that a session is part of the catalogs of both events in the record that is retained. 

Note: it seems there are some sessions that occur multiple times in the data set but are not included in both event catalogs. This seems to be just some form of data pollution.

Let's first see how many events are part of both events' catalogs. 

In [175]:
# The code feature is supposed to be the unique identifier of sessions
# Let's see at the multifold occurrence of individual code values

counts = ss['code'].value_counts()
counts.head(13)

TRN4077    4
TRN4088    4
TRN4092    4
TRN5774    4
TRN4631    4
TRN4082    4
TRN4019    4
TRN5793    4
TRN4081    4
TRN4084    4
TRN4065    4
TRN5777    4
BUS2128    2
Name: code, dtype: int64

We have found quite a few code values that occur multiple times in the data frame. Each code should be in the data frame only once. Let's further look into these sessions.

In [176]:
# let's create a data frame with all sessions whose code occurs more than one in the data frame
duplicates = ss[ss['code'].isin(counts.index[counts > 1])]
# show the first ten records of these 'candidate duplicates'
duplicates[['code','title','event','catalog'  ]].sort_values('code').head(10)

,code,title,event,catalog
147,BOF4977,A Cloud- and Container-Based Approach to Micro...,Oracle OpenWorld,oow
151,BOF4977,A Cloud- and Container-Based Approach to Micro...,Oracle OpenWorld,codeone
148,BOF5232,Introducing Agile into the US Air Force in 12 ...,Oracle OpenWorld,oow
166,BOF5232,Introducing Agile into the US Air Force in 12 ...,Oracle OpenWorld,codeone
149,BOF5714,Istio and Envoy: Enabling Sidecars for Microse...,Oracle OpenWorld,oow
167,BOF5714,Istio and Envoy: Enabling Sidecars for Microse...,Oracle OpenWorld,codeone
146,BOF6086,Accelerating Big Data Analytics on Oracle Cloud,Oracle OpenWorld,oow
150,BOF6086,Accelerating Big Data Analytics on Oracle Cloud,Oracle OpenWorld,codeone
290,BUS1224,DBA Does Docker,Oracle OpenWorld,oow
361,BUS1224,DBA Does Docker,Oracle OpenWorld,codeone


In [177]:
#are duplicates indeed associated with both events?
# some are - but not all of them:
duplicates.loc[duplicates['code']=='BOF4977']


,abstract,attributevalues,catalog,code,event,eventCode,files,length,participants,sponsors,times,title,title_sort,type
147,Microservices are independent—sure. Complex tr...,"[{'value': 'Intermediate', 'attributevalue_id'...",oow,BOF4977,Oracle OpenWorld,oow18,"[{'fileId': '1540345180625001DgUR', 'filename'...",45.0,[{'speakerId': '14574694420080015I8e_150393630...,None,"[{'sessionTimeID': '1535418245131001BYdX', 'se...",A Cloud- and Container-Based Approach to Micro...,acloudandcontainerbasedapproac,Birds of a Feather (BOF) Session
151,Microservices are independent—sure. Complex tr...,"[{'value': 'Intermediate', 'attributevalue_id'...",codeone,BOF4977,Oracle OpenWorld,oow18,"[{'fileId': '1540345180625001DgUR', 'filename'...",45.0,[{'speakerId': '14574694420080015I8e_150393630...,None,"[{'sessionTimeID': '1535418245131001BYdX', 'se...",A Cloud- and Container-Based Approach to Micro...,acloudandcontainerbasedapproac,Birds of a Feather (BOF) Session


The next step is a little bit complex: we want to record the fact that certain sessions (actually session codes) are associated with both catalogs. We join the sessions in `ss` with the sessions that occur multiple times in `duplicates` and we join records in `ss` with their counterparts (same session code) that have a different catalog origin.

This gives us a data frame with all session codes associated with both catalogs.

In [178]:
# find all sessions that occur in both catalogs: set their catalog attribute to both
# set catalog="both" if session in duplicates with a different catalog value than the session's own catalog value
doubleCatalogSessions = pd.merge(ss, duplicates, on=['code'], how='inner').query('catalog_y != catalog_x')
doubleCatalogSessions[['code','catalog_x', 'catalog_y']] .head(20)



,code,catalog_x,catalog_y
45,TRN5793,oow,codeone
47,TRN5793,oow,codeone
48,TRN5793,codeone,oow
50,TRN5793,codeone,oow
53,TRN5793,oow,codeone
55,TRN5793,oow,codeone
56,TRN5793,codeone,oow
58,TRN5793,codeone,oow
137,TRN4084,oow,codeone
139,TRN4084,oow,codeone


The master dataset is still `ss`. All sessions in this data frame whose session code appears in `doubleCatalogSessions` will get their *catalog* attribute updated to *both*. The cell will then show the values in catalog and the number of their occurrences.

In [179]:
# all sessions in doubleCatalogSessions occur in both oow and code one session catalog
# time to update column catalog for all sessions in ss that have a code that occurs in doubleCatalogSessions['code']
ss.loc[ss['code'].isin(doubleCatalogSessions['code']),'catalog']='both'
ss['catalog'].value_counts()

oow        1592
both        234
codeone     165
Name: catalog, dtype: int64

If we now drop any duplicate records - any sessions whose session code occurs more than once - we will reduce our data frame to the unique set of sessions that actually took place, without the ghost duplicates introduced in our data set because a session appeared in more than one catalog.

In [180]:
#Drop duplicates - identifying rows by their code
ss.drop_duplicates(subset=['code'], keep='first', inplace=True)

In [181]:
#hpw many sessions appear in each and in both catalogs?
ss['catalog'].value_counts()

oow        1470
codeone     165
both        105
Name: catalog, dtype: int64

<a name="explore" />

# Exploring the Data
Let's briefly look at the data we now have in the Pandas Data Frame. What does the data look like? What values do we have in the columns? And what complexity is hiding in some columns with nested values, such as participants, attributevalues and files.

Note: https://morphocode.com/pandas-cheat-sheet/ provides a quick overview of commands used for inspecting and manipulating the data frame.

In [182]:
#an overview of the current contents of the data frame
ss.head(6)


,abstract,attributevalues,catalog,code,event,eventCode,files,length,participants,sponsors,times,title,title_sort,type
0,What if you could benefit from new database qu...,"[{'value': 'Beginner', 'attributevalue_id': '1...",oow,TRN4070,Oracle OpenWorld,oow18,"[{'fileId': '1540594881063001PjdE', 'filename'...",45.0,[{'speakerId': '14574694377280015QNA_150393630...,None,"[{'sessionTimeID': '1532616475210001usPa', 'se...",A Database Proxy for Transparent High Availabi...,adatabaseproxyfortransparenthi,Product Training Session
1,How does a real-world performance engineer int...,"[{'value': 'Beginner', 'attributevalue_id': '1...",oow,TRN4026,Oracle OpenWorld,oow18,"[{'fileId': '1540053090863001JeFM', 'filename'...",45.0,[{'speakerId': '145746969078000151fg_150393630...,None,"[{'sessionTimeID': '1531945554973001z4Hi', 'se...",A Day in the Life of a Real-World Performance ...,adayinthelifeofarealworldperfo,Product Training Session
2,Understand the tools and techniques available ...,"[{'value': 'Beginner', 'attributevalue_id': '1...",oow,TRN4567,Oracle OpenWorld,oow18,"[{'fileId': '1539988618000001VcCe', 'filename'...",45.0,[{'speakerId': '1468532302119001DtNf_150393630...,None,"[{'sessionTimeID': '15352384203600012G04', 'se...",Best Practices for Diagnosing and Troubleshoot...,bestpracticesfordiagnosingandt,Product Training Session
3,In order to derive value from the Internet of ...,"[{'value': 'Product Training Session', 'attrib...",oow,TRN6363,Oracle OpenWorld,oow18,"[{'fileId': '1539982921144001RL4h', 'filename'...",45.0,[{'speakerId': '1500580326390001qTtd_150393630...,None,"[{'sessionTimeID': '1535223010412001BrNh', 'se...",Oracle Internet of Things Cloud Service and En...,oracleinternetofthingscloudser,Product Training Session
4,Oracle Linux powers tens of thousands of enter...,"[{'value': 'Beginner', 'attributevalue_id': '1...",oow,TRN5828,Oracle OpenWorld,oow18,"[{'fileId': '1540152941675001kZHw', 'filename'...",45.0,[{'speakerId': '14574694616420015a7T_150393630...,None,"[{'sessionTimeID': '1535040668257001twFq', 'se...",Oracle Linux and Oracle VM: Get Trained for Cl...,oraclelinuxandoraclevmgettrain,Product Training Session
5,"What do Oracle Database listener, dedicated se...","[{'value': 'Beginner', 'attributevalue_id': '1...",oow,TRN4073,Oracle OpenWorld,oow18,None,45.0,[{'speakerId': '14574694719240015Rek_150393630...,None,"[{'sessionTimeID': '1531848510829001f1BX', 'se...",Oracle Net Services: Best Practices for Databa...,oraclenetservicesbestpractices,Product Training Session


In [183]:
# and this overview of the rows and columns in the data frame.
print(ss.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1740 entries, 0 to 1990
Data columns (total 14 columns):
abstract           1740 non-null object
attributevalues    1740 non-null object
catalog            1740 non-null object
code               1740 non-null object
event              1740 non-null object
eventCode          1740 non-null object
files              1171 non-null object
length             1740 non-null float64
participants       1740 non-null object
sponsors           189 non-null object
times              1739 non-null object
title              1740 non-null object
title_sort         1740 non-null object
type               1740 non-null object
dtypes: float64(1), object(13)
memory usage: 203.9+ KB
None


In [184]:
#let's look at all different values for length (the duration of each session in minutes)
ss['length'].unique()
# yes - it is that simple!

array([ 45.,  20., 240., 180.,  90., 120.,  75.,  60.,  30., 105.,  50.,
        25.,  55.,  15.])

In [185]:
# and what about (session) type?
ss['type'].unique()

array(['Product Training Session', 'Birds of a Feather (BOF) Session',
       'Burning Questions Session', 'Business Use Case Session',
       'Customer Case Study Session', 'Developer Session',
       'Executive Solution Session', 'Flipped Session', 'General Session',
       'HOL (Hands-on Lab) Session', 'Home Room Session',
       'Ignite Session', 'Keynote Session', 'Meet the Experts',
       'Product Keynote Session', 'Product Overview Session',
       'Product Roadmap Session', 'Special Interest Group (SIG) Meeting',
       'Theater Session', 'Thought Leader Session',
       'Tips and Tricks Session', 'Tutorial Session'], dtype=object)

Some of the columns in the data frame contain complex, nested values. For example the `attributevalues` column. It contains a JSON array - a list of objects that each describe some attribute for the session. Examples of session attributes that are defined in this somewhat convoluted wat are *(target Experience) Level*, *Track*, *Day* , *Role*. A little later on, we will create new, proper features in the data frame based on values extracted from this complex attributevalues colunmn - in order to make it possible to make good use of this information for visualization, analysis and machine learning.

In [186]:
# some of the columns or attributes have nested values. It seems useful to take a closer look at them. 
# show nested array attributevalues for first record in Dataframe - this is an attribute which contains an array of objects that each define a specific characteristic of the session, 
# such as date and time, track, topic, level, role, company market size, 

ss['attributevalues'][10]

[{'value': 'Beginner',
  'attributevalue_id': '1486682599762002N05j',
  'rf_attributevalue_id': '1486682600237001NDhX',
  'attributevalue_code': '1486682599762002N05j',
  'dataForm': '1503936308L8ef50df64',
  'published': 1.0,
  'viewAccessPublic': True,
  'viewAccess': ['1533140082245001une2',
   '1533141196635001EZdJ',
   '1533143193464001uQsQ',
   '1533146325902001u9v6'],
  'scheduleAccess': ['1533141196635001EZdJ', '1533143193464001uQsQ'],
  'attribute_id': 'SessionsbyExperienceLevel',
  'attribute': 'Sessions by Experience Level',
  'displayorder': 1.0},
 {'value': 'Intermediate',
  'attributevalue_id': '1486682599762003NG3N',
  'rf_attributevalue_id': '1486682600237002N7TO',
  'attributevalue_code': '1486682599762003NG3N',
  'dataForm': '1503936308L8ef50df64',
  'published': 1.0,
  'viewAccessPublic': True,
  'viewAccess': ['1533140082245001une2',
   '1533141196635001EZdJ',
   '1533143193464001uQsQ',
   '1533146325902001u9v6'],
  'scheduleAccess': ['1533141196635001EZdJ', '153314

The *participants* column also contains a complex object. This column too contains a JSON array with the people associated with a session as speaker. The array contains a nested object for each speaker. This object has a lot of data in it - from name and biography and special designations (titles) for the speaker to company, job title, URL to a picture and details on all (other) sessions the speaker is involved in.

Let's take a look at an example.

In [187]:
#show nested array participants for 11th record in the Dataframe
ss['participants'][10]


[{'speakerId': '1500578268510001KsST_150393630796000oow18',
  'userRef': '1500578268510001KsST',
  'firstName': 'Ron',
  'lastName': 'Soltani',
  'bio': 'My name is Ron Soltani, and I am a Sr. Principal Instructor at Oracle University.  I have been with Oracle University for over 14 Years teaching, Oracle Database, MYSQL, and Times Ten Database courses and I am the lead instructor for Oracle Database New Features, Oracle Database Security, and MYSQL Administration courses.  I am based in Southern Oregon and enjoy spending my spare time outdoors and working on our farm. ',
  'companyName': 'Oracle',
  'jobTitle': 'Senior Principal Instructor',
  'photoURL': 'https://static.rainfocus.com/oracle/oow18/att/1500578268510001KsST/attprofile/Ron_Soltani_Headshot_1538595194670001C3jN.jpg',
  'Speaker-Photo-Published': 'Published',
  'globalFirstname': 'Ron',
  'globalLastname': 'Soltani',
  'globalFullName': 'Ron Soltani',
  'globalBio': 'My name is Ron Soltani, and I am a Sr. Principal Instruc

The *files* column too contains a JSON array. This array contains entries for files associated with the session. These files provide the slides or other supporting content for the session. Each session is supposed to have exactly one files associated with it. Some do not - for example because the speaker(s) forgot to upload their slides. 

Let's inspect the JSON contents of the *files* column. It contains the name of the file and the URL from where it can be downloaded.

In [188]:
#show nested array files
ss['files'][0]

[{'fileId': '1540594881063001PjdE',
  'filename': 'TRN4070_A-Database-Proxy-for-Transparent-High-Availability-Performance-Routing-and-Security.pdf',
  'url': 'https://static.rainfocus.com/oracle/oow18/sess/1523650128750001VI1i/PF/TRN4070_A-Database-Proxy-for-Transparent-High-Availability-Performance-Routing-and-Security_1540594881049001PwHW.pdf',
  'viewFileAccess': []}]

<a name="enriching" />

# Enriching the data
In this section we are engineering the data to produce some attributes or features that are easier to work with once we start doing data science activities such as business intelligence or machine learning. In this section we are not yet actually bringing in external data sources to add information we not already have in our set. However, we are making the data we already have more accessible and thereby more valuable. So in that sense, this too can be called *enriching*. 

The enrichments performed on the session data in the data frame :
* set a flag at session level to indicate whether the session is an Oracle session (with Oracle staff among the speakers- speaker has Oracle in companyName attribute)
* set a flag at session level to indicate whether a file has been uploaded for the session
* set a flag at session level to indicate whether one of the speakers has Java Rockstar, Java Champion, Developer Champion/Groundbreaker Ambassador, ACE or ACE Director
* set attribute level (based on attributevalues array) - beginner, all, ...
* derive track(s?) from attributevalues array (where attribute_id=Track or attribute_id=CodeOneTracks  )
* set attribute with number of speakers on session
* number of instances of the session    

### Oracle Speaker and File Uploaded
These functions `oracle_speaker` and `file_flag` are invoked for every record in the data frame. They are used to derive new, first class attributes to indicate whether or not at least one speaker working for Oracle is associated with a session (Y or N) and if a file has been upload (presumably with slides) for the session. The information represented by these two new attributes already exists in the data frame - but in a way that makes it quite unaccessible to the data analyst.

In [189]:
#function to derive Oracle flag from participants
def oracle_speaker(session):
      result = "N"
      # loop over speakers; if for any of them, companyName contains the word Oracle, the result =Y  
      for x in session["participants"][:]:
        if ("oracle" in x.get('companyName','x').lower()):
          result='Y'
      return result

In [190]:
#function to derive file flag from files
def file_flag(session):
      result = "N"
      if type(session.get("files",None)) =='list' :
          # loop over files; if any exist, then result = Y
          for x in session["files"][:]:
             if x['fileId']:
               result='Y'
      return result

New columns `oracle_speaker` and `file_flag` are added to the data frame with values derived for each record by applying the functions with corresponding names.  

In [191]:
#set oracle_speaker flag
#apply function oracle_speaker to every row in the data frame to derive values for the new column oracle_speaker
ss['oracle_speaker'] = ss.apply(oracle_speaker, axis=1)

#set file_flag
#apply function file_flag to every row in the data frame to derive values for the new column file_flag
ss['file_flag'] = ss.apply(file_flag, axis=1)


# show the values and the number of occurrences for the new column oracle_speaker
ss['oracle_speaker'].value_counts()


Y    1320
N     420
Name: oracle_speaker, dtype: int64

### Speaker Designations
Many of the speakers are special - in the sense that they have been awarded community awards and titles, such as (Oracle) Java Champion, Oracle ACE Directory, JavaOne Rockstar and Groundbreaker Ambassador. These designations can be found for speakers (a nested JSON object) in their *attributevalues* feature - which happens to be another nested JSON object.

The next function *speaker_designation* finds out for a session if it has at least one speaker associated with it who has the requested designation.

In [192]:
#function to derive designation flag from speakers
# values for designation: JavaOne Rockstar, Oracle ACE Director, Oracle Java Champion, Groundbreaker Ambassador, 
def speaker_designation(session, designation):
      result = "N"
      # loop over speakers and their attributevalues; if any exist with attribute_id == specialdesignations and value == JavaOne Rockstar
      for x in session["participants"][:]:
          if "attributevalues" in x: 
              for y in x["attributevalues"][:]:
                if "attribute_id" in y: 
                    if y["attribute_id"]=="specialdesignations":
                        if y["value"]== designation:
                            result="Y"
      return result

The next cell iterates over four major `designations` and derives for each designation a new column in the data frame that contain Y or N, depending on whether a speaker with the designation will present in the session. 

In [193]:
#set flags for designations
designations = ['JavaOne Rockstar', 'Oracle ACE Director', 'Oracle Java Champion', 'Groundbreaker Ambassador']
for d in designations:
   ss[d] = ss.apply(speaker_designation, args=(d,), axis=1) 

Let's check the newly created columns: what values do they contain and how often does each value occur?

In [194]:
ss[['JavaOne Rockstar','Oracle ACE Director' , 'Oracle Java Champion', 'Groundbreaker Ambassador']].apply(pd.value_counts).fillna(0)

,JavaOne Rockstar,Oracle ACE Director,Oracle Java Champion,Groundbreaker Ambassador
N,1676,1651,1692,1675
Y,64,89,48,65


### Level
Each session can be described as suitable for one or more levels of experience: Beginner, Intermediate or Advaned. Each session can be associated with all levels - or a subset of them. This level indication is somewhat hidden away, in the attributevalues object. The next function `session_level` will unearth for a given session whether it is associated with the specified level, Y or N.

In [195]:
#function to derive level flag for a session
def session_level(session, level):
      result = "N"
      # loop over attributevalues; if any exist with attribute_id == "SessionsbyExperienceLevel", and value == level
      for x in session["attributevalues"][:]:
        if "attribute_id" in x: 
            if x["attribute_id"]=="SessionsbyExperienceLevel":
                if x["value"]== level:
                    result="Y"
                    break # no point in continuing if we have found what we are looking for
      return result

This cell runs through all three level values and creates a new column in the data frame for each level. It will set a Y or N for each session in the new columns, depending on whether session is associated with the level, or not.

In [196]:
#set flags for designations
levels = ['Intermediate', 'Beginner', 'Advanced']
for l in levels:
   ss[l] = ss.apply(session_level, args=(l,), axis=1) 
print("Assigned Level Flags (Advanced, Intermediate, Beginner)")

Assigned Level Flags (Advanced, Intermediate, Beginner)


Next we will derive values for a new column 'All' that indicates whether a session has been associated with all levels - even though I am not sure what exactly that means.

In [197]:
def isAll(session):
    return 'Y' if session['Beginner'] == 'Y'  and session['Intermediate'] == 'Y' and session['Advanced'] == 'Y'else 'N'

ss['All'] = ss.apply( isAll, axis=1) 

In [198]:
ss[['Intermediate', 'Beginner', 'Advanced', 'All']].apply(pd.value_counts).fillna(0)

,Intermediate,Beginner,Advanced,All
Y,1470,1317,1110,966
N,270,423,630,774


### Track
All sessions are assigned to one or more tracks. These tracks are categories that help attendees identify and assess sessions. Some examples of tracks are: Core Java Platform, Development Tools, Oracle Cloud Platform, MySQL, Containers, Serverless, and Cloud, Emerging Technologies, Modern Web, Application Development, Infrastructure Technologies (Data Center).

Depending on whether a session originates from the Oracle OpenWorld or CodeOne catalog, the track(s) are found in the nested object *attributevalues* under the attribute_id *CodeOneTracks* or just *Track*.

The next function is created to return a String array for a session with all the tracks associated with the session.

In [199]:
#function to derive track flag for a session
def session_track(session):
      result = ['abc']      
      # loop over attributevalues; if any exist with attribute_id == "SessionsbyExperienceLevel", and value == level
      for x in session["attributevalues"][:]:
        if "attribute_id" in x: 
            if x["attribute_id"]=="CodeOneTracks":
               result.append( x["value"])
            if x["attribute_id"]=="Track":               
               result.append( x["value"])
      del result[0]   
      return result

The next cell uses the function `session_track` to produce the value for each session for the track. The cell then prints out the tracks for a sample of sessions.

In [200]:
# add column track with a value derived from the session record
ss["track"] = ss.apply(session_track,  axis=1) 
print("Assigned Track")
ss[['title','catalog','track']].tail(10)

Assigned Track


,title,catalog,track
1979,Revisiting Effective Java in 2018,codeone,"[Core Java Platform, Development Tools]"
1980,Setting Up Asynchronous Systems with Fn Flow,codeone,"[Containers, Serverless, and Cloud]"
1981,The Java Platform Module System in Action: Bui...,codeone,"[Core Java Platform, Development Tools]"
1983,Clouds in My Coffee: Java on Mobile for iOS an...,codeone,"[Containers, Serverless, and Cloud, Java Ecosy..."
1984,Getting Started with Ethereum Private Blockchain,codeone,[Emerging Technologies]
1985,GraphQL in the Java World: Let’s Go for a Dive,codeone,"[Emerging Technologies, Modern Web]"
1986,"Java 5, 6, 7, 8, 9, 10, 11: What Did You Miss?",codeone,[Core Java Platform]
1987,Kubernetes for Java Developers,codeone,"[Containers, Serverless, and Cloud, Developmen..."
1988,Let's Start Java11 from the Beginning with JShell,codeone,[Core Java Platform]
1990,Preventing Errors Before They Happen: The Chec...,codeone,[Development Tools]


### Number of Speakers per Session
The number of speakers making an appearance in a session is another fact that while not readily available is also hiding in our data frame. We will turn this piece of information into an explicit feature. The next cell counts the number of elements in the  participants array - and assigns it to the speaker_count attribute of each session record.

In [201]:
#set number of speakers for each session by taking the length of the list in the participants column
ss["speaker_count"] = ss['participants'].apply(lambda x: len(x))
# list the values in the new speaker_count column and the number of occurrences. 12 participants in one session?
ss['speaker_count'].value_counts()

1     749
2     508
3     357
4      67
5      33
8      10
6       9
7       4
12      1
11      1
10      1
Name: speaker_count, dtype: int64

### Number of instances per session
Most sessions are scheduled just once. However, some sessions are executed multiple times. This can be derived from the *times* column in the data frame, simply by taking the number of elements in the JSON array in that column.

The next cell adds a column to the data frame, with for each session the number of times that session takes place.

In [202]:
#set number of instancesfor each session by taking the length of the list in the times column
ss["instance_count"] = ss['times'].apply(lambda x: len(x) if x else None)
ss['instance_count'].value_counts()

1.0    1685
2.0      39
4.0       9
3.0       5
8.0       1
Name: instance_count, dtype: int64

### Session Room Capacity
The session records have a *times* feature - a JSON array with all the instances of the session. The elements contain the *capacity* attribute that gives the size or capcity of the room in which the session is scheduled. Because most sessions occur only once it seems acceptable to set a room_capacity feature on all session records in the data frame derived from the capacity found first element in the times feature. Likewise, we can derive the value for room itself.

In [203]:
#set the room capacity based on the capacity of the room of the first entry in the times list
# note: when the session is scheduled multiple times, not all rooms may have the same capacity; that detail gets lost
ss["room_capacity"]= ss['times'].apply(lambda x:x[0]['capacity'] if x else None)
ss["room"]= ss['times'].apply(lambda x:x[0]['room'] if x else None)

### Session Slot - Day and Time
As discussed in the preceding section, the session records have a *times* feature - a JSON array with all the instances of the session. The elements contain attributes *dayName* and *time* that mark the slot in which the session is scehduled. Because most sessions occur only once it seems acceptable to set a session day and time feature from the values for time and day found first element in the times feature. Likewise, we can derive the Python DateTime value for the starting timestamp of the session.

In [204]:
#likewise derive day, time and room - from the first occurrence of the session
ss["day"]= ss['times'].apply(lambda x:x[0]['dayName'] if x else None)
ss["time"]= ss['times'].apply(lambda x:x[0]['time'] if x else None)

ss[['day','time']].apply(pd.value_counts).fillna(0)

,day,time
08:00,0.0,3.0
08:30,0.0,3.0
08:45,0.0,15.0
09:00,0.0,129.0
09:30,0.0,4.0
10:00,0.0,80.0
10:30,0.0,55.0
10:45,0.0,18.0
11:00,0.0,79.0
11:05,0.0,1.0


The columns `day` and `time` are just strings. It may be useful for the data analysts further downstream in the data science pipeline to also have a real DateTime object to work with. The next cell introduces the `session_timestamp` column, set to the real timestamp derived from day and time. Note that we make use of one external piece of data not found in the datalake: the fact that Sunday in this case means 21st October 2018.  

In [205]:
import datetime
# see https://stackabuse.com/converting-strings-to-datetime-in-python/

#Sunday means 21st October 2018; monday through thursday are the days following the 21st
dayMap = {'Sunday': '21', 'Monday': '22', 'Tuesday':'23', 'Wednesday':'24', 'Thursday':'25'}


def create_timestamp(day, timestr):
    if not timestr:
        return None
    if not day:
        return None
    dtString = '2018-10-'+ dayMap[day] +' '+ timestr
    return datetime.datetime.strptime(dtString, '%Y-%m-%d %H:%M')

def create_session_timestamp(session):
    return create_timestamp(session['day'], session['time'])

ss['session_timestamp'] = ss.apply(create_session_timestamp, axis=1) 

In [206]:
ss[['day','time','session_timestamp'] ].head(10)

,day,time,session_timestamp
0,Wednesday,11:15,2018-10-24 11:15:00
1,Monday,15:45,2018-10-22 15:45:00
2,Monday,11:30,2018-10-22 11:30:00
3,Monday,12:30,2018-10-22 12:30:00
4,Monday,10:30,2018-10-22 10:30:00
5,Monday,15:45,2018-10-22 15:45:00
6,Tuesday,11:15,2018-10-23 11:15:00
7,Monday,17:45,2018-10-22 17:45:00
8,Monday,17:45,2018-10-22 17:45:00
9,Thursday,12:00,2018-10-25 12:00:00


<a name="publish" />

## Persist Pandas Dataframe - as single, consolidated, reshaped, enriched JSON file

One thing we want to be able to do with the data we gather, is to persist it for future use by data analists, data scientists and other stakeholders. We could store the data in a NoSQL database, a cloud storage service or simply as a local file. For now, let's do the latter: store the cleansed and reshaped data in a local JSON file for further enrichment, visualization and machine learning purposes.

The file is called `oow2018-sessions-wrangled.json` and it will be stored in the `datawarehouse` folder.

In [207]:
dataWarehouse = "datawarehouse/" # file system directory used for storing the wrangled data
ss.to_json("{0}oow2018-sessions-wrangled.json".format(dataWarehouse), force_ascii=False)

A quick check to see whether the wrangled session data was successfully written to disk - and can be read again. If we can read it, than we can safely assume that in the next phase in the data analytics flow the same will succeed.

In [208]:
dubss = pd.read_json("{0}oow2018-sessions-wrangled.json".format(dataWarehouse))
dubss.head(10)

,abstract,attributevalues,catalog,code,event,eventCode,files,length,participants,sponsors,...,Advanced,All,track,speaker_count,instance_count,room_capacity,room,day,time,session_timestamp
0,What if you could benefit from new database qu...,"[{'value': 'Beginner', 'attributevalue_id': '1...",oow,TRN4070,Oracle OpenWorld,oow18,"[{'fileId': '1540594881063001PjdE', 'filename'...",45,[{'speakerId': '14574694377280015QNA_150393630...,None,...,Y,Y,"[- Application Development, Oracle Cloud Platf...",3,1.0,308.0,Moscone West - Room 3009,Wednesday,11:15,1.540380e+12
1,How does a real-world performance engineer int...,"[{'value': 'Beginner', 'attributevalue_id': '1...",oow,TRN4026,Oracle OpenWorld,oow18,"[{'fileId': '1540053090863001JeFM', 'filename'...",45,[{'speakerId': '145746969078000151fg_150393630...,None,...,Y,Y,"[- Application Development, Oracle Cloud Platf...",2,1.0,313.0,Moscone West - Room 3004,Monday,15:45,1.540223e+12
10,In this session learn how to make client data ...,"[{'value': 'Beginner', 'attributevalue_id': '1...",oow,TRN6449,Oracle OpenWorld,oow18,"[{'fileId': '1540334126918001DZZj', 'filename'...",45,[{'speakerId': '1500578268510001KsST_150393630...,None,...,Y,Y,[],1,1.0,314.0,Moscone West - Room 3002,Tuesday,15:45,1.540310e+12
100,In this session learn how to transfer your st...,"[{'value': 'Product Training Session', 'attrib...",oow,TRN6527,Oracle OpenWorld,oow18,None,20,[{'speakerId': '14664511836130012Qxc_150393630...,None,...,N,N,[],1,1.0,75.0,The Exchange @ Moscone South - OU Mini Theater,Wednesday,16:25,1.540398e+12
1000,Are you using service-oriented architecture ap...,"[{'value': 'Intermediate', 'attributevalue_id'...",oow,PRO4370,Oracle OpenWorld,oow18,"[{'fileId': '1541663350742001gmaq', 'filename'...",45,[{'speakerId': '14574694357570045XMa_150393630...,None,...,Y,N,"[- Management Cloud, Featured Customers, Your ...",3,1.0,494.0,Marriott Marquis (Golden Gate Level) - Golden ...,Monday,11:30,1.540208e+12
1001,Oracle Integration Cloud and Oracle Java Cloud...,"[{'value': 'Beginner', 'attributevalue_id': '1...",oow,PRO4478,Oracle OpenWorld,oow18,"[{'fileId': '15404077399430016Rwl', 'filename'...",45,[{'speakerId': '14574694420840025lYB_150393630...,None,...,Y,Y,"[- Application Development, - Integration, - M...",3,1.0,416.0,Marriott Marquis (Golden Gate Level) - Golden ...,Tuesday,13:45,1.540302e+12
1002,Are you using Oracle Enterprise Manager for da...,"[{'value': 'Beginner', 'attributevalue_id': '1...",oow,PRO4281,Oracle OpenWorld,oow18,"[{'fileId': '1541619601519001X5OH', 'filename'...",45,[{'speakerId': '14574694365080015kH7_150393630...,None,...,Y,Y,"[- Management Cloud, SMB Solutions, Oracle Clo...",1,1.0,416.0,Marriott Marquis (Golden Gate Level) - Golden ...,Wednesday,12:30,1.540384e+12
1003,This session covers Oracle Management Cloud de...,"[{'value': 'Beginner', 'attributevalue_id': '1...",oow,PRO4448,Oracle OpenWorld,oow18,"[{'fileId': '1540853870696001NQh6', 'filename'...",45,[{'speakerId': '14574696980920015zaB_150393630...,None,...,Y,Y,"[- Management Cloud, SMB Solutions, Your Cloud...",2,1.0,494.0,Marriott Marquis (Golden Gate Level) - Golden ...,Monday,10:30,1.540204e+12
1004,"As IT infrastructure expands to new IaaS, PaaS...","[{'value': 'Beginner', 'attributevalue_id': '1...",oow,PRO4494,Oracle OpenWorld,oow18,None,45,[{'speakerId': '1467407457306001EUm3_150393630...,None,...,Y,Y,"[- Identity and Security, - Management Cloud, ...",3,1.0,416.0,Marriott Marquis (Golden Gate Level) - Golden ...,Thursday,12:00,1.540469e+12
1005,Proper integration of your fulfillment applica...,"[{'value': 'Beginner', 'attributevalue_id': '1...",oow,PRO4206,Oracle OpenWorld,oow18,"[{'fileId': '15410024797530014IjR', 'filename'...",45,[{'speakerId': '145746946465200359QN_150393630...,None,...,Y,Y,"[Intelligent Cloud Applications, - Oracle E-Bu...",2,1.0,170.0,Moscone North - Room 20,Wednesday,15:45,1.540396e+12


If and when the previous cell lists session records correctly, then the data warehouse has been populated with the consolidated file *oow2018-sessions-wrangled.json* with all sessions - cleansed, deduplicated, enriched and ready for further processing.

The wrangled data set no longer contains many of the attributes not conceivably useful for further analysis. It has been extended with (derived) attributes that will probably be useful for next data analytics tasks. Additionally, the record set has been deduplicated to only the uniqe sessions.   

In [209]:
# a quick list of all columns
dubss.columns

Index(['abstract', 'attributevalues', 'catalog', 'code', 'event', 'eventCode',
       'files', 'length', 'participants', 'sponsors', 'times', 'title',
       'title_sort', 'type', 'oracle_speaker', 'file_flag', 'JavaOne Rockstar',
       'Oracle ACE Director', 'Oracle Java Champion',
       'Groundbreaker Ambassador', 'Intermediate', 'Beginner', 'Advanced',
       'All', 'track', 'speaker_count', 'instance_count', 'room_capacity',
       'room', 'day', 'time', 'session_timestamp'],
      dtype='object')